In [7]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import pandas as pd
import cv2
import argparse
import mediapipe as mdp

In [15]:
# GETS THE REQUIRED ARGUMENTS IN CLI 

# def get_args():
#     parser=argparse.ArgumentParser()

#     parser.add_argument("--device", type= int,default=0)
#     parser.add_argument("--staticimage", action="store_true")
#     parser.add_argument("--width",help="Maximun width", type= int,default=960)
#     parser.add_argument("--height",help="Maximun height", type= int,default=540)

#     parser.add_argument("--minDetection" ,help="Minimum confidence for detection",type=float,default=0.6)
#     parser.add_argument("--minTracking" ,help="Minimum confidenxce for tracking",type=float,default=0.8)

#     args = parser.parse_args()

#     return args


In [22]:
def load_models(use_static_image_mode,min_detection,min_tracking):

#LOAD MODEL TO DETECT LANDMARKS IN HANDS
    mdp_hands=mdp.solutions.hands
    hands=mdp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_hands_num=2,
        min_detection_confidence=min_detection,
        min_tracking_confidence=min_tracking
        )


In [34]:
def camera_input(cam_device,cam_width,cam_height):

#POPERLY INITIALIZE CAMERA FOR WORKING
    cam=cv2.VideoCapture(cam_device)
    cam.set(cv2.CAP_PROP_FRAME_WIDTH,cam_width)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT,cam_height)
    while True:
        ret, frame = cam.read()

        if not ret:
            print("Error: Failed to capture frame.")
            break

        # Display the frame
        cv2.imshow('Camera Feed', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    print(cam)
    cam.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()



< cv2.VideoCapture 000001DA8F030470>


In [37]:
def main():
    
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    cam_device=0
    cam_width=960
    cam_height=540
# GET THE CAMERA ARGUMENTS FROM CLI COMMANDS
    use_static_image_mode=True
    min_detection=0.6
    min_tracking=0.8

    camera_input(cam_device,cam_width,cam_height)
    load_models(use_static_image_mode,min_detection,min_tracking)
    
main()

TypeError: camera_input() takes 0 positional arguments but 3 were given